In [5]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
#

In [6]:
#read original data source
df=pd.read_csv('../data/NYC_Building_Energy_and_Water_Data_Disclosure_for_Local_Law_84__2022-Present__20250106.csv')


In [7]:
df_info = pd.read_csv(
    "../data/NYC_Community_Boards.csv",
    usecols=[
        
        "Community Board 1",
      
    ],
)
cbList=df_info["Community Board 1"].unique().tolist()

In [8]:
#keep property with Multifamilty housign as primary usage
df_housing=df[df['Primary Property Type - Portfolio Manager-Calculated']=='Multifamily Housing'].reset_index().drop(columns='index')
#transform the list of property usage into a python list
df_housing['HousingAllPropUseCount']=df_housing['List of All Property Use Types (GFA) (ft²)'].apply(lambda x: len(x.split(',')))

In [9]:
#keep the property Multifamilty Housign only - usage list with 1 element only
df_housingOnly=df_housing[df_housing['HousingAllPropUseCount']==1]

In [10]:
#trasform not available data in detectable python missing data
df_housingOnly.replace({'Not Available':np.nan}, inplace=True)
#keep the property with available location and ENERGY STAR Score.
df_housingOnly.dropna(subset=['Latitude','Longitude','ENERGY STAR Score'],inplace=True)

In [11]:
#drop the full empty columns -e.g. meaningful for non-housing usage only
df_housingOnly.dropna(how='all', axis=1, inplace=True)

In [12]:
#detect properties with duplicated entries for a given year- few cases of double submission (e.g. fix submitted data) - go with the most recent
df_housingOnly.drop_duplicates(subset=['Property ID','Calendar Year','NYC Building Identification Number (BIN)','NYC Borough, Block and Lot (BBL)'],keep='last',inplace=True)
df_housingOnly.reset_index().drop(columns='index', inplace=True)

In [13]:
#add calculated col concatenating Borough and Community Board Number - for viz purpose
df_housingOnly['Borough_CommBoard']=df_housingOnly['Borough']+"_"+df_housingOnly['Community Board'].astype(int).astype('str')

In [14]:
df_housingOnly=df_housingOnly[df_housingOnly['Community Board'].isin(cbList)]

In [15]:
#export the final CSV

df_housingOnly.to_csv('../data/NYC_housingOnly_v0.csv', index=False)